In [1]:
import json
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from bs4 import BeautifulSoup
import requests
import re
import operator
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\casey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\casey\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\casey\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\casey\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


pip install nltk

pip install beautifulsoup4

pip install spacy

python -m spacy download en_core_web_sm

In [2]:
nlp = en_core_web_sm.load()

In [3]:
with open('gg2020.json', encoding='utf8') as f:
    lines = f.readlines()

gg_data = []
for line in lines:
    gg_data.append(json.loads(line))
    

In [35]:
people_awards = ["Best Performance by an Actress in a Motion Picture - Drama", 
          "Best Performance by an Actor in a Motion Picture - Drama", 
          "Best Performance by an Actress in a Motion Picture - Musical or Comedy",
         "Best Performance by an Actor in a Motion Picture - Musical or Comedy",
         "Best Performance by an Actress in a Supporting Role in any Motion Picture",
         "Best Performance by an Actor in a Supporting Role in any Motion Picture",
         "Best Director - Motion Picture",
         "Best Screenplay - Motion Picture",]
awards = ['cecil b. demille award', 
          'best motion picture - drama', 
          'best performance by an actress in a motion picture - drama', 
          'best performance by an actor in a motion picture - drama', 
          'best motion picture - comedy or musical', 
          'best performance by an actress in a motion picture - comedy or musical', 
          'best performance by an actor in a motion picture - comedy or musical', 
          'best animated feature film', 
          'best foreign language film', 
          'best performance by an actress in a supporting role in a motion picture', 
          'best performance by an actor in a supporting role in a motion picture', 
          'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best original song - motion picture', 'best television series - drama', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best television series - comedy or musical', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best mini-series or motion picture made for television', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture made for television', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television'].sort



<built-in method sort of list object at 0x0000020964176E88>


In [45]:

motion_picture_awards = [
'Best Motion Picture - Drama',
'Best Motion Picture - Musical or Comedy',
'Best Motion Picture - Foreign Language',
'Best Motion Picture - Animated',
'Best Director - Motion Picture',
'Best Actor - Motion Picture Drama',
'Best Actor - Motion Picture Musical or Comedy',
'Best Actress - Motion Picture Drama',
'Best Actress - Motion Picture Musical or Comedy',
'Best Supporting Actor - Motion Picture',
'Best Supporting Actress - Motion Picture',
'Best Screenplay - Motion Picture',
'Best Original Score - Motion Picture',
'Best Original Song - Motion Picture',
'Cecil B. DeMille Award for Lifetime Achievement in Motion Pictures']

tv_awards = [
'Best Television Series - Drama',
'Best Television Series - Musical or Comedy',
'Best Miniseries or Television Film',
'Best Actor - Television Series Drama',
'Best Actor - Television Series Musical or Comedy',
'Best Actor - Miniseries or Television Film',
'Best Actress - Television Series Drama',
'Best Actress - Television Series Musical or Comedy',
'Best Actress - Miniseries or Television Film',
'Best Supporting Actor - Series, Miniseries or Television Film',
'Best Supporting Actress - Series, Miniseries or Television Film',
'Carol Burnett Award for Lifetime Achievement in Television']

all_awards = motion_picture_awards + tv_awards
print(all_awards)

['Best Motion Picture - Drama', 'Best Motion Picture - Musical or Comedy', 'Best Motion Picture - Foreign Language', 'Best Motion Picture - Animated', 'Best Director - Motion Picture', 'Best Actor - Motion Picture Drama', 'Best Actor - Motion Picture Musical or Comedy', 'Best Actress - Motion Picture Drama', 'Best Actress - Motion Picture Musical or Comedy', 'Best Supporting Actor - Motion Picture', 'Best Supporting Actress - Motion Picture', 'Best Screenplay - Motion Picture', 'Best Original Score - Motion Picture', 'Best Original Song - Motion Picture', 'Cecil B. DeMille Award for Lifetime Achievement in Motion Pictures', 'Best Television Series - Drama', 'Best Television Series - Musical or Comedy', 'Best Miniseries or Television Film', 'Best Actor - Television Series Drama', 'Best Actor - Television Series Musical or Comedy', 'Best Actor - Miniseries or Television Film', 'Best Actress - Television Series Drama', 'Best Actress - Television Series Musical or Comedy', 'Best Actress - 

In [5]:
tweets_with_host = []
for tweet in gg_data: 
    tweet_text = tweet['text']
    if("host" in tweet_text.lower()) :
        tweets_with_host.append(tweet)
len(tweets_with_host)

5146

In [6]:
def get_names(text):
    article = nlp(text)
    labels = [x.label_ for x in article.ents]
    [(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(text) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]
    parts_of_speech = dict([(str(x), x.label_) for x in nlp(text).ents])
    names = []
    for (key, value) in parts_of_speech.items() :
        if(value == "PERSON") :
            names.append(key)
    return names 

In [7]:
def get_host_name():
    host_name_count = {}
    for tweet in tweets_with_host: 
        names = get_names(tweet["text"])
        for name in names :
            if name in host_name_count :
                host_name_count[name] = host_name_count[name] + 1
            else :
                host_name_count[name] = 1
    host = max(host_name_count.items(), key=operator.itemgetter(1))[0]
    print(host)
    return host
get_host_name()

Ricky Gervais


'Ricky Gervais'

In [62]:
# returns a list of tweets containting the given string 
def get_tweets(s):
    tweets_with_str = []
    for tweet in gg_data: 
        tweet_text = tweet['text']
        if(s.lower() in tweet_text.lower()) :
            tweets_with_str.append(tweet)
    return tweets_with_str

def get_tweets_text(s):
    tweets_with_str = []
    for tweet in gg_data: 
        tweet_text = tweet['text']
        if(s.lower() in tweet_text.lower()) :
            tweets_with_str.append(tweet['text'])
    return tweets_with_str

In [58]:
#filter list of tweet bodies for exps
def filter_tweets(tweets, exps):
    tweets_with_str = []
    for tweet in tweets:
        tweet_lower = tweet['text'].lower()
        for exp in exps:
            t = re.findall(exp,tweet_lower)
            print(t)
            break
    print(tweets_with_str)
    return tweets_with_str

In [28]:
# awards_list is a list of award strings ["Best Motion Picture - Drama", "Best Director", etc]
# assuming winners are the most name dropped with an award, this can also be extended to find nominees
# returns map of award : winner (hopefully)
def get_award_winners(awards_list):
    award_winners = {}
    for award in awards_list:
        tweets_with_award = get_tweets(award)
        name_count = {}
        for tweet in tweets_with_award: 
            names = get_names(tweet["text"])
            for name in names :
                if name.lower() == "goldenglobes" or name.lower() == "golden globes" or len(name.split()) < 2:
                    pass
                elif name in name_count :
                    name_count[name] = name_count[name] + 1
                else :
                    name_count[name] = 1
        winner = max(name_count.items(), key=operator.itemgetter(1))[0]
        award_winners[award] = winner
    return award_winners

get_award_winners(people_awards)

{'Best Performance by an Actress in a Motion Picture - Drama': 'Renée Zellweger',
 'Best Performance by an Actor in a Motion Picture - Drama': 'Joaquin Phoenix',
 'Best Performance by an Actress in a Motion Picture - Musical or Comedy': 'Ana de Armas',
 'Best Performance by an Actor in a Motion Picture - Musical or Comedy': 'Leonardo DiCaprio',
 'Best Performance by an Actress in a Supporting Role in any Motion Picture': 'Laura Dern',
 'Best Performance by an Actor in a Supporting Role in any Motion Picture': 'Brad Pitt',
 'Best Director - Motion Picture': 'Sam Mendes',
 'Best Screenplay - Motion Picture': 'Quentin Tarantino'}

In [64]:
def get_award_presenters(awards_list):
    single_presenter_search = re.compile(r'[A-Z][a-z]+\s[A-Z][a-z]+(?=\spresent)')
    two_presenters_search1 = re.compile(r'[A-Z][a-z]+\s[A-Z][a-z]+\sand\s[A-Z][a-z]+\s[A-Z][a-z]+present')
    two_presenters_search2 = re.compile(r'[A-Z][a-z]+\s[A-Z][a-z]+\sand\s[A-Z][a-z]+\s[A-Z][a-z]+introduc')
    filters = [single_presenter_search, two_presenters_search1, two_presenters_search2]

    award_to_presenters = {}
    for award in all_awards:
        award_lower = award.lower()
        #get tweets with award name
        tweets = get_tweets_text(award_lower)
        print(tweets)
        #filter tweets for presenting format
        #tweets = filter_tweets(tweets, filters)
        break
        
            
        award_to_presenters[award] = predicted_presenter
    return award_to_presenters

get_award_presenters(awards)

['Congratulations to all the Golden Globe Award Winners!  \nTechnicolor and our creative studios are honored to have worked on 4 of the winning projects, including the Best Motion Picture - Drama winner, 1917.\n@technicolorpost @mpc_film #postproduction #colorfinishing #goldenglobes https://t.co/CXmrJdEnw2', 'Congratulations to 1917 for its two Golden Globe wins last night - Best Director and Best Motion Picture - Drama\n\nExperience the spectacular 1917 from Friday - tickets on sale now at https://t.co/nC4I1su6zJ https://t.co/OQVMlk3l9V', 'Golden Globes recap!\n\nBest Motion Picture - Drama\n🌎1917\n\nBest Motion Picture - Comedy\n🌎Once Upon a Time in Hollywood\n\nBest Animated Feature Film\n🌎Missing Link\n\nBest Foreign Language Film\n🌎Parasite', "Congratulations to the following clients who were part of these fantastic Golden Globe Winners 2020; Billy Postlethwaite for '1917' Golden Globe for Best Motion Picture - Drama https://t.co/G2qRm4r4NM", 'It was a big and surprising win for #

{}

In [10]:
text = gg_data[610]['text']
article = nlp(text)
labels = [x.label_ for x in article.ents]
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                  for y
                                  in nlp(text) 
                                  if not y.is_stop and y.pos_ != 'PUNCT']]
parts_of_speech = dict([(str(x), x.label_) for x in nlp(text).ents])
print(parts_of_speech)

{'Awkwafina': 'PERSON', 'last night': 'TIME', 'Farewell': 'ORG'}


In [18]:
ne_tree = ne_chunk(pos_tag(word_tokenize(gg_data[0]["text"])))
print(ne_tree)

(S
  •/JJ
  (ORGANIZATION Priyanka/NNP Chopra/NNP)
  and/CC
  (PERSON Nick/NNP Jonas/NNP)
  at/IN
  the/DT
  77th/CD
  (PERSON Annual/NNP Golden/NNP Globe/NNP Awards/NNP)
  present/VBD
  the/DT
  award/NN
  for/IN
  (GPE Best/NNP)
  TV/NN
  series/NN
  for/IN
  a/DT
  musical/JJ
  or/CC
  comedy/NN
  •/JJ
  #/#
  goldenglobes2020/JJ
  @/NNP
  (PERSON Golden/NNP Globes/NNP)
  https/NN
  :/:
  //t.co/fivnKsE9Ib/NN)


In [19]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [20]:
sent = preprocess(gg_data[0]["text"])
sent

[('•', 'JJ'),
 ('Priyanka', 'NNP'),
 ('Chopra', 'NNP'),
 ('and', 'CC'),
 ('Nick', 'NNP'),
 ('Jonas', 'NNP'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('77th', 'CD'),
 ('Annual', 'NNP'),
 ('Golden', 'NNP'),
 ('Globe', 'NNP'),
 ('Awards', 'NNP'),
 ('present', 'VBD'),
 ('the', 'DT'),
 ('award', 'NN'),
 ('for', 'IN'),
 ('Best', 'NNP'),
 ('TV', 'NN'),
 ('series', 'NN'),
 ('for', 'IN'),
 ('a', 'DT'),
 ('musical', 'JJ'),
 ('or', 'CC'),
 ('comedy', 'NN'),
 ('•', 'JJ'),
 ('#', '#'),
 ('goldenglobes2020', 'JJ'),
 ('@', 'NNP'),
 ('Golden', 'NNP'),
 ('Globes', 'NNP'),
 ('https', 'NN'),
 (':', ':'),
 ('//t.co/fivnKsE9Ib', 'NN')]

In [21]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [22]:
cp = nltk.RegexpParser(pattern)
cs = cp.parse(sent)
print(cs)

(S
  •/JJ
  Priyanka/NNP
  Chopra/NNP
  and/CC
  Nick/NNP
  Jonas/NNP
  at/IN
  the/DT
  77th/CD
  Annual/NNP
  Golden/NNP
  Globe/NNP
  Awards/NNP
  present/VBD
  (NP the/DT award/NN)
  for/IN
  Best/NNP
  (NP TV/NN)
  (NP series/NN)
  for/IN
  a/DT
  musical/JJ
  or/CC
  (NP comedy/NN)
  •/JJ
  #/#
  goldenglobes2020/JJ
  @/NNP
  Golden/NNP
  Globes/NNP
  (NP https/NN)
  :/:
  (NP //t.co/fivnKsE9Ib/NN))


In [23]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [24]:
doc = nlp(gg_data[0]["text"])
pprint([(X.text, X.label_) for X in doc.ents])

NameError: name 'pprint' is not defined

In [ ]:
pprint([(X, X.ent_iob_, X.ent_type_) for X in doc])

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [ ]:
first_tweet = gg_data[0]["text"]
article = nlp(first_tweet)

In [ ]:
labels = [x.label_ for x in article.ents]
Counter(labels)

In [ ]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(first_tweet) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

In [ ]:
dict([(str(x), x.label_) for x in nlp(first_tweet).ents])